In [1]:
import openai

In [2]:
from datasets import load_dataset

In [3]:
train_dataset = load_dataset("orieg/elsevier-oa-cc-by", split="train")

No config specified, defaulting to: elsevier-oa-cc-by/all
Found cached dataset elsevier-oa-cc-by (/home/user/.cache/huggingface/datasets/orieg___elsevier-oa-cc-by/all/1.0.1/90990052f835613074d87af3592fd8eaee912f17ca4d0401bcf6d2d791d45117)


In [4]:
print(train_dataset)

Dataset({
    features: ['title', 'abstract', 'subjareas', 'keywords', 'asjc', 'body_text', 'author_highlights'],
    num_rows: 32072
})


In [5]:
print(train_dataset[0]['body_text'])
print('\n')
print(train_dataset[0]['abstract'])

['Autonomous systems are increasingly required in various practical applications, including unmanned aircraft, driver-less cars, healthcare robots, manufacturing robots, etc.', 'In all of these cases it is easy to imagine a situation where an autonomous system causes harm to people or property, as a result of an error in its engineering, or an unfortunate combination of circumstances.', 'Therefore, if such autonomous systems are to operate within society, we must be able to trust that their behaviour complies with the legal, social, and ethical norms of that society.', 'Determining the trustworthiness of technology in this respect is usually delegated to a regulatory body, such as the Federal Aviation Administration (for aircraft in the USA) or the Vehicle Certification Authority (for road vehicles in the UK).', 'The process is known as certification, and is used to determine the safety and reliability of safety-critical technology, including aircraft, road vehicles, nuclear reactors, 

In [6]:
def prepare_data(example):
    article = ''
    for line in example['body_text']:
        article += line + ' '
    example['prompt'] = article[:1100]                  # limit length to fit model
    example['completion'] = example['abstract'][:400]   # limit length to fit model
    return example

In [7]:
train_dataset = train_dataset.map(prepare_data, remove_columns=['title', 'abstract', 'subjareas', 'keywords', 'asjc', 'body_text', 'author_highlights'])

  0%|          | 0/32072 [00:00<?, ?ex/s]

In [8]:
print(train_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 32072
})


In [9]:
print(train_dataset[0]['prompt'])
print(train_dataset[0]['completion'])

Autonomous systems are increasingly required in various practical applications, including unmanned aircraft, driver-less cars, healthcare robots, manufacturing robots, etc. In all of these cases it is easy to imagine a situation where an autonomous system causes harm to people or property, as a result of an error in its engineering, or an unfortunate combination of circumstances. Therefore, if such autonomous systems are to operate within society, we must be able to trust that their behaviour complies with the legal, social, and ethical norms of that society. Determining the trustworthiness of technology in this respect is usually delegated to a regulatory body, such as the Federal Aviation Administration (for aircraft in the USA) or the Vehicle Certification Authority (for road vehicles in the UK). The process is known as certification, and is used to determine the safety and reliability of safety-critical technology, including aircraft, road vehicles, nuclear reactors, pharmaceutical

In [10]:
import json

In [11]:
# for demonstration only 10 articles selected (should be at least 200 for decent results) 
finetune_dataset = train_dataset.select(range(10))

In [12]:
finetune_dataset.to_json("finetune_dataset_json", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

15450

In [13]:
import os

In [14]:
os.environ['OPENAI_API_KEY'] = "<OPENAI_KEY>"

In [15]:
!openai api fine_tunes.create -t "finetune_dataset_json" -m "davinci" --suffix "davin-else"

Upload progress: 100%|████████████████████| 15.4k/15.4k [00:00<00:00, 19.8Mit/s]
Uploaded file from finetune_dataset_json: file-RST2GrKiCQ1aWt3iVx3V8eD6
Created fine-tune: ft-AWNZQTkindHZgRIUUrO5Wtmr
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-02-13 21:14:55] Created fine-tune: ft-AWNZQTkindHZgRIUUrO5Wtmr

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-AWNZQTkindHZgRIUUrO5Wtmr



In [17]:
!openai api fine_tunes.follow -i ft-AWNZQTkindHZgRIUUrO5Wtmr

[2023-02-13 21:14:55] Created fine-tune: ft-AWNZQTkindHZgRIUUrO5Wtmr
[2023-02-13 21:22:21] Fine-tune costs $0.39
[2023-02-13 21:22:26] Fine-tune enqueued. Queue number: 16
[2023-02-13 21:23:08] Fine-tune is in the queue. Queue number: 15
[2023-02-13 21:23:09] Fine-tune is in the queue. Queue number: 14
[2023-02-13 21:23:21] Fine-tune is in the queue. Queue number: 13
[2023-02-13 21:26:27] Fine-tune is in the queue. Queue number: 12
[2023-02-13 21:27:08] Fine-tune is in the queue. Queue number: 11
[2023-02-13 21:29:59] Fine-tune is in the queue. Queue number: 10
[2023-02-13 21:30:06] Fine-tune is in the queue. Queue number: 9
[2023-02-13 21:32:48] Fine-tune is in the queue. Queue number: 8
[2023-02-13 21:37:08] Fine-tune is in the queue. Queue number: 7
[2023-02-13 21:38:52] Fine-tune is in the queue. Queue number: 6
[2023-02-13 21:39:26] Fine-tune is in the queue. Queue number: 5
[2023-02-13 21:40:21] Fine-tune is in the queue. Queue number: 4
[2023-02-13 21:41:49] Fine-tune is in the 

In [ ]:
# now prompt model to summarize article with chunks as described in notebook Summarize_with_chunks_GPT3.ipynb
# including:
# openai.Completion.create(
#     model=FINE_TUNED_MODEL,
#     prompt=PROMPT)